Data preparation has been performed in data_prep.py, and stored in the relative filepath under `../../data/audio_data_train.npz` for training data and `../../data/audio_data_test.npz` for testing data respectively.


In [1]:
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

uploaded_testing = files.upload()

Mounted at /content/drive


Saving fullstop_prediction_test.npz to fullstop_prediction_test.npz


In [6]:
!ls

drive  fullstop_prediction_test.npz  sample_data


In [8]:
uploaded_training = files.upload()

Saving fullstop_prediction_train.npz to fullstop_prediction_train.npz


In [10]:
!ls

drive  fullstop_prediction_test.npz  fullstop_prediction_train.npz  sample_data


In [24]:
# Load training data
train_data = np.load('fullstop_prediction_train.npz')
X_train = train_data['X_train'] # 893 sets of training data with dimensions (216, 1025)
y_train = train_data['y_train'] # 893 labels

# Load testing data
test_data = np.load('fullstop_prediction_test.npz')
X_test = test_data['X_test'] # 224 sets of test data with dimensions (216, 1025)
y_test = test_data['y_test'] # 224 labels

In [25]:
print(f'X_train for item 0: {X_train[0]}')
print(f'y_train for item 0: {y_train[0]}')

X_train for item 0: [[-50.59104  -39.94433  -37.29682  ... -80.       -80.       -80.      ]
 [-50.642223 -36.894035 -33.203125 ... -80.       -80.       -80.      ]
 [-46.85834  -34.352818 -30.948872 ... -80.       -80.       -80.      ]
 ...
 [-62.434986 -36.93872  -32.473602 ... -80.       -80.       -80.      ]
 [-51.505238 -38.332172 -32.605656 ... -77.064    -77.06359  -77.06343 ]
 [-57.762917 -41.996998 -35.549538 ... -65.66542  -65.665436 -65.665375]]
y_train for item 0: 0


In [26]:
X_train = X_train[..., np.newaxis]  # Shape: (893, 216, 1025, 1)
X_test = X_test[..., np.newaxis]    # Shape: (224, 216, 1025, 1)


In [27]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(216, 1025, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (0 or 1)
])


In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [29]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32
)


Epoch 1/20
 4/28 [===>..........................] - ETA: 5:03 - loss: 33.8920 - accuracy: 0.4766

KeyboardInterrupt: 